In [47]:
import os
import pandas as pd
import json

current_dir = os.getcwd()
root_dir = os.path.dirname(os.path.dirname(current_dir))
_RAWFIRESPATH = f"{root_dir}/RawData/Historical_FiresRAW"

In [9]:
_year = 2005
_file = f"{_RAWFIRESPATH}/{_year}/2005-01-01_37.7388357_-8.750062888101874.csv"
df = pd.read_csv(f"{_file}")
print(df.head())

   latitude  longitude  generationtime_ms  utc_offset_seconds timezone  \
0  37.71529  -8.783417           0.598907                   0      GMT   

  timezone_abbreviation  elevation hourly_units.time  \
0                   GMT       50.0           iso8601   

  hourly_units.temperature_2m hourly_units.relative_humidity_2m  ...  \
0                          °C                                 %  ...   

                    hourly.soil_moisture_28_to_100cm  \
0  [0.224, 0.224, 0.224, 0.224, 0.224, 0.224, 0.2...   

                   hourly.soil_moisture_100_to_255cm  \
0  [0.206, 0.206, 0.206, 0.206, 0.206, 0.206, 0.2...   

                                       hourly.is_day  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, ...   

                            hourly.sunshine_duration  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                  hourly.shortwave_radiation_instant  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                     hourly.

In [76]:
_year = 2005
_file = f"{_RAWFIRESPATH}/{_year}/2005-01-01_37.7388357_-8.750062888101874.csv"
df = pd.read_csv(f"{_file}")
# print(df.head())
column_names = df.columns.tolist()
print(column_names)

# Get all column names
column_names = df.columns.tolist()

# Find columns that contain 'hourly_units'
columns_to_remove = [col for col in column_names if 'hourly_units' in col]
columns_to_remove.extend(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation'])

# Remove the columns
df = df.drop(columns=columns_to_remove)

# Print the updated column names
updated_column_names = df.columns.tolist()
print(updated_column_names)


['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly_units.time', 'hourly_units.temperature_2m', 'hourly_units.relative_humidity_2m', 'hourly_units.dew_point_2m', 'hourly_units.apparent_temperature', 'hourly_units.precipitation', 'hourly_units.rain', 'hourly_units.snowfall', 'hourly_units.snow_depth', 'hourly_units.weather_code', 'hourly_units.pressure_msl', 'hourly_units.surface_pressure', 'hourly_units.cloud_cover', 'hourly_units.cloud_cover_low', 'hourly_units.cloud_cover_mid', 'hourly_units.cloud_cover_high', 'hourly_units.et0_fao_evapotranspiration', 'hourly_units.vapour_pressure_deficit', 'hourly_units.wind_speed_10m', 'hourly_units.wind_speed_100m', 'hourly_units.wind_direction_10m', 'hourly_units.wind_direction_100m', 'hourly_units.wind_gusts_10m', 'hourly_units.soil_temperature_0_to_7cm', 'hourly_units.soil_temperature_7_to_28cm', 'hourly_units.soil_temperature_28_to_100cm', 'hourly_units.soil_temperatur

In [13]:
print(df.head())

   elevation                                        hourly.time  \
0       50.0  ['2005-01-01T00:00', '2005-01-01T01:00', '2005...   

                               hourly.temperature_2m  \
0  [7.8, 7.4, 7.2, 6.8, 6.4, 6.0, 6.3, 6.5, 7.3, ...   

                         hourly.relative_humidity_2m  \
0  [94, 94, 95, 96, 97, 98, 97, 95, 95, 97, 90, 8...   

                                 hourly.dew_point_2m  \
0  [6.9, 6.6, 6.5, 6.2, 5.9, 5.7, 5.9, 5.8, 6.5, ...   

                         hourly.apparent_temperature  \
0  [5.6, 5.2, 5.1, 4.6, 4.0, 3.7, 4.2, 4.5, 5.7, ...   

                                hourly.precipitation  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                         hourly.rain  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                     hourly.snowfall  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                   hourly.snow_depth  ...  \
0  [0.0, 0.0,

In [87]:
import ast

# Convert the string representation of a list into a list
list_from_string = ast.literal_eval(df['hourly.time'].iloc[0])[15]

print(list_from_string)

#print(list_from_string[15])

# Print the length of the list
#print(len(list_from_string))

IndexError: single positional indexer is out-of-bounds

In [21]:
print(df['hourly.time'].iloc[0].)

['2005-01-01T00:00', '2005-01-01T01:00', '2005-01-01T02:00', '2005-01-01T03:00', '2005-01-01T04:00', '2005-01-01T05:00', '2005-01-01T06:00', '2005-01-01T07:00', '2005-01-01T08:00', '2005-01-01T09:00', '2005-01-01T10:00', '2005-01-01T11:00', '2005-01-01T12:00', '2005-01-01T13:00', '2005-01-01T14:00', '2005-01-01T15:00', '2005-01-01T16:00', '2005-01-01T17:00', '2005-01-01T18:00', '2005-01-01T19:00', '2005-01-01T20:00', '2005-01-01T21:00', '2005-01-01T22:00', '2005-01-01T23:00']


In [17]:
df['hourly.time'].apply(len)

0    480
Name: hourly.time, dtype: int64

In [77]:
_file_radiation = _file.replace(".csv", "_radiation.csv")
df = pd.read_csv(f"{_file_radiation}")
print(df.head())

df = pd.read_csv(f"{_file_radiation}")
# print(df.head())
column_names = df.columns.tolist()
print(column_names)

# Get all column names
column_names = df.columns.tolist()

# Find columns that contain 'hourly_units'
columns_to_remove = [col for col in column_names if 'hourly_units' in col]
columns_to_remove.extend(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly.time', 'hourly.global_tilted_irradiance_instant', 'hourly.diffuse_radiation_instant', 'hourly.terrestrial_radiation_instant', 'hourly.direct_radiation_instant', 'hourly.shortwave_radiation_instant', 'hourly.direct_normal_irradiance_instant'])

# Remove the columns
df = df.drop(columns=columns_to_remove)

# Print the updated column names
updated_column_names_radiation = df.columns.tolist()
print(updated_column_names_radiation)


   latitude  longitude  generationtime_ms  utc_offset_seconds timezone  \
0  37.71529  -8.783417           0.172973                   0      GMT   

  timezone_abbreviation  elevation hourly_units.time  \
0                   GMT       50.0           iso8601   

  hourly_units.shortwave_radiation hourly_units.direct_radiation  ...  \
0                             W/m²                          W/m²  ...   

                            hourly.diffuse_radiation  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                     hourly.direct_normal_irradiance  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                     hourly.global_tilted_irradiance  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                        hourly.terrestrial_radiation  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                  hourly.shortwave_radiation_instant  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                     hourl

In [39]:
# Convert the lists to sets
set1 = set(updated_column_names_radiation)
set2 = set(updated_column_names)

# Find the common elements
common_elements = set1 & set2

# Print the common elements
print(common_elements)

set()


In [78]:
header = ['year', 'date', 'district', 'municipality', 'parish', 'latitude', 'longitude']

columns = header + updated_column_names + updated_column_names_radiation


df = pd.DataFrame(columns=columns)

df.to_csv('FireDataset.csv', index=False)

In [43]:
print(len(columns))

87


In [79]:
#columns = header + updated_column_names + updated_column_names_radiation
print(updated_column_names, updated_column_names_radiation)

['elevation', 'hourly.time', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.dew_point_2m', 'hourly.apparent_temperature', 'hourly.precipitation', 'hourly.rain', 'hourly.snowfall', 'hourly.snow_depth', 'hourly.weather_code', 'hourly.pressure_msl', 'hourly.surface_pressure', 'hourly.cloud_cover', 'hourly.cloud_cover_low', 'hourly.cloud_cover_mid', 'hourly.cloud_cover_high', 'hourly.et0_fao_evapotranspiration', 'hourly.vapour_pressure_deficit', 'hourly.wind_speed_10m', 'hourly.wind_speed_100m', 'hourly.wind_direction_10m', 'hourly.wind_direction_100m', 'hourly.wind_gusts_10m', 'hourly.soil_temperature_0_to_7cm', 'hourly.soil_temperature_7_to_28cm', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_temperature_100_to_255cm', 'hourly.soil_moisture_0_to_7cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.soil_moisture_28_to_100cm', 'hourly.soil_moisture_100_to_255cm', 'hourly.is_day', 'hourly.sunshine_duration', 'hourly.shortwave_radiation_instant', 'hourly.direct_radiation_in

In [116]:
_year = 2023

df = pd.read_csv("FireDataset.csv")

print(df.head())

dfsourcex = pd.read_csv(f'{_RAWFIRESPATH}/filtered_icnf_2023.csv')
filtered_df = dfsourcex[(dfsourcex['TIPO'] == 'Florestal') & (dfsourcex['TIPOCAUSA'].isin(['Desconhecida', 'Natural']))]
print(len(filtered_df))

second_row = filtered_df.iloc[0]
DIA = second_row['DIA']
MES = second_row['MES']
HORA = second_row['HORA']
LAT = second_row['LAT']
LON = second_row['LON']
ANO = second_row['ANO']
DISTRICTO = second_row['DISTRITO']
CONCELHO = second_row['CONCELHO']
FREGUESIA = second_row['FREGUESIA']

MeteorologyFile = False
RadiationFile = False

print(DIA, MES, HORA, LAT, LON, ANO)

MES = "0" + str(MES) if MES < 10 else str(MES)
DIA = "0" + str(DIA) if DIA < 10 else str(DIA)

_fileName = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}.csv.csv"

_fileName_radiation = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}_radiation.csv"

print(_fileName, _fileName_radiation)

isExistMeteorology = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName}')
if(isExistMeteorology):
    MeteorologyFile = True

isExistRadiation = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName_radiation}')
if(isExistRadiation):
    RadiationFile = True

print(isExistMeteorology, RadiationFile)

if(MeteorologyFile and RadiationFile):
    dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName}")
    dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName_radiation}")

    print(dfMet['elevation'].iloc[0])

    list_from_string = ast.literal_eval(dfMet[updated_column_names[2]].iloc[0])[HORA]

    print(list_from_string)

    list_from_string = ast.literal_eval(dfMet[updated_column_names[1]].iloc[0])[HORA]

    print(list_from_string)

    new_data = {
        'year': ANO,
        'date': f"{ANO}-{MES}-{DIA}",
        'district': DISTRICTO,
        'municipality': CONCELHO,
        'parish': FREGUESIA,
        'latitude': LAT,
        'longitude': LON,
        'elevation': dfMet['elevation'].iloc[0]
    }

    try:
        for i in range(1, len(updated_column_names)):
            new_data[updated_column_names[i]] = ast.literal_eval(dfMet[updated_column_names[i]].iloc[0])[HORA]
    except Exception as e:
        print(e)

    try:
        for i in range(0, len(updated_column_names_radiation)):
            new_data[updated_column_names_radiation[i]] = ast.literal_eval(dfRad[updated_column_names_radiation[i]].iloc[0])[HORA]
    except Exception as e:
        print(e)

    new_df = pd.DataFrame(new_data, index=[0])

    new_df.to_csv('FireDataset.csv', mode='a', header=False, index=False)


   year        date district  municipality    parish   latitude  longitude  \
0  2023  2023-01-21     Faro  Castro Marim  Odeleite  37.320278  -7.486111   

   elevation       hourly.time  hourly.temperature_2m  ...  \
0       83.0  2023-01-21T13:00                   17.7  ...   

   hourly.diffuse_radiation_instant  hourly.direct_normal_irradiance_instant  \
0                             140.1                                    571.7   

   hourly.global_tilted_irradiance_instant  \
0                                    448.4   

   hourly.terrestrial_radiation_instant  hourly.shortwave_radiation  \
0                                 761.8                       448.0   

   hourly.direct_radiation  hourly.diffuse_radiation  \
0                    308.0                     140.0   

   hourly.direct_normal_irradiance  hourly.global_tilted_irradiance  \
0                            571.7                            448.0   

   hourly.terrestrial_radiation  
0                         761.0

In [114]:
df = pd.read_csv("FireDataset.csv")
second_row = df.iloc[0]
print(second_row['hourly.terrestrial_radiation_instant'])

#print(df.head())

761.8


In [123]:
_year = 2023

df = pd.read_csv("FireDataset.csv")
dfsourcex = pd.read_csv(f'{_RAWFIRESPATH}/filtered_icnf_{_year}.csv')
filtered_df = dfsourcex[(dfsourcex['TIPO'] == 'Florestal') & (dfsourcex['TIPOCAUSA'].isin(['Desconhecida', 'Natural']))]
print(len(filtered_df))

for r in range(0, len(filtered_df)):

    second_row = filtered_df.iloc[r]
    DIA = second_row['DIA']
    MES = second_row['MES']
    HORA = second_row['HORA']
    LAT = second_row['LAT']
    LON = second_row['LON']
    ANO = second_row['ANO']
    DISTRICTO = second_row['DISTRITO']
    CONCELHO = second_row['CONCELHO']
    FREGUESIA = second_row['FREGUESIA']

    MeteorologyFile = False
    RadiationFile = False

    print(DIA, MES, HORA, LAT, LON, ANO)

    MES = "0" + str(MES) if MES < 10 else str(MES)
    DIA = "0" + str(DIA) if DIA < 10 else str(DIA)

    _fileName = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}.csv.csv"

    _fileName_radiation = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}_radiation.csv"

    print(_fileName, _fileName_radiation)

    isExistMeteorology = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName}')
    if(isExistMeteorology):
        MeteorologyFile = True
    else:
        with open('missing_files.txt', 'a') as f:
            f.write(_fileName + '\n')
        
        continue

    isExistRadiation = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName_radiation}')
    if(isExistRadiation):
        RadiationFile = True
    else:
        with open('missing_files.txt', 'a') as f:
            f.write(_fileName+ '\n')
        
        continue

    print(isExistMeteorology, RadiationFile)

    if(MeteorologyFile and RadiationFile):
        try:
            dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName}")
            dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName_radiation}")

            print(dfMet['elevation'].iloc[0]) # este 0 refere-se ao dfMet

            list_from_string = ast.literal_eval(dfMet[updated_column_names[2]].iloc[0])[HORA]

            print(list_from_string)

            list_from_string = ast.literal_eval(dfMet[updated_column_names[1]].iloc[0])[HORA]

            print(list_from_string)

            new_data = {
                'year': ANO,
                'date': f"{ANO}-{MES}-{DIA}",
                'district': DISTRICTO,
                'municipality': CONCELHO,
                'parish': FREGUESIA,
                'latitude': LAT,
                'longitude': LON,
                'elevation': dfMet['elevation'].iloc[0]
            }

            try:
                for i in range(1, len(updated_column_names)):
                    new_data[updated_column_names[i]] = ast.literal_eval(dfMet[updated_column_names[i]].iloc[0])[HORA]
            except Exception as e:
                print(e)

            try:
                for i in range(0, len(updated_column_names_radiation)):
                    new_data[updated_column_names_radiation[i]] = ast.literal_eval(dfRad[updated_column_names_radiation[i]].iloc[0])[HORA]
            except Exception as e:
                print(e)

            new_df = pd.DataFrame(new_data, index=[0])
            new_df.to_csv('FireDataset.csv', mode='a', header=False, index=False)
        except Exception as e:
            print(e)
            with open('missing_files.txt', 'a') as f:
                f.write(_fileName + '\n')
            
            continue

    

1935
21 1 13 37.3202777777778 -7.48611111111111 2023
21_01_13_37.3202777777778_-7.48611111111111.csv.csv 21_01_13_37.3202777777778_-7.48611111111111_radiation.csv
True True
83.0
17.7
2023-01-21T13:00
27 1 12 41.2671949999862 -8.31583799997965 2023
27_01_12_41.2671949999862_-8.31583799997965.csv.csv 27_01_12_41.2671949999862_-8.31583799997965_radiation.csv
True True
230.0
10.7
2023-01-27T12:00
27 1 23 41.1159549999237 -8.1857329999076 2023
27_01_23_41.1159549999237_-8.1857329999076.csv.csv 27_01_23_41.1159549999237_-8.1857329999076_radiation.csv
True True
405.0
2.6
2023-01-27T23:00
28 1 17 40.4325 -8.36777777777778 2023
28_01_17_40.4325_-8.36777777777778.csv.csv 28_01_17_40.4325_-8.36777777777778_radiation.csv
True True
224.0
9.8
2023-01-28T17:00
29 1 14 38.2088888888889 -7.8175 2023
29_01_14_38.2088888888889_-7.8175.csv.csv 29_01_14_38.2088888888889_-7.8175_radiation.csv
True True
193.0
12.0
2023-01-29T14:00
29 1 22 41.7005555555556 -8.51833333333333 2023
29_01_22_41.7005555555556_-8.5

In [129]:
with open('missing_files.txt', 'r') as f:
    mf = f.readlines()
    

print(mf)
dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{mf[2].strip()}")
print(dfMet.head())

dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{mf[2].strip().replace('.csv.csv', '_radiation.csv')}")


['02_07_16_40.6336111111111_-8.08472222222222.csv.csv\n', '02_07_16_40.6336111111111_-8.08472222222222.csv.csv\n', '12_08_16_38.7592599995931_-9.21870999998517.csv.csv\n']
   latitude  longitude  generationtime_ms  utc_offset_seconds timezone  \
0  38.76977   -9.20874           0.398993                   0      GMT   

  timezone_abbreviation  elevation hourly_units.time  \
0                   GMT       92.0           iso8601   

  hourly_units.temperature_2m hourly_units.relative_humidity_2m  ...  \
0                          °C                                 %  ...   

                    hourly.soil_moisture_28_to_100cm  \
0  [0.204, 0.204, 0.204, 0.204, 0.204, 0.204, 0.2...   

                   hourly.soil_moisture_100_to_255cm  \
0  [0.299, 0.299, 0.299, 0.299, 0.299, 0.299, 0.2...   

                                       hourly.is_day  \
0  [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   

                            hourly.sunshine_duration  \
0  [0.0, 0.0, 0.0, 0.0, 0.

FileNotFoundError: [Errno 2] No such file or directory: '/home/ori/Desktop/SPAWN/FirePrediction/RawData/Historical_FiresRAW/2023/12_08_16_38.7592599995931_-9.21870999998517_radiation.csv'

In [133]:
import requests
_year = 2023
print(mf[2].strip().replace('.csv.csv', ''))
gerrad = mf[2].strip().replace('.csv.csv', '').split('_')
DIA = gerrad[0]
MES = gerrad[1]
HORA = gerrad[2]
LAT = gerrad[3]
LON = gerrad[4]

print(DIA, MES, HORA, LAT, LON)

_DATA = f"{ANO}-{MES}-{DIA}"


_fileName = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}_radiation"

        
isExist = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName}.csv')
if(isExist):
    print(f'File already exists')
else:
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={LAT}&longitude={LON}&start_date={_DATA}&end_date={_DATA}&hourly=shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant"
    #url = f"https://archive-api.open-meteo.com/v1/archive?latitude={LAT}&longitude={LON}&start_date={_DATA}&end_date={_DATA}&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,is_day,sunshine_duration,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant&timezone=GMT"
    response = requests.get(url)
    print(response.status_code)
    data = response.json()
    df = pd.json_normalize(data)
    # Save the DataFrame to a CSV file
    df.to_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName}.csv", index=False)


12_08_16_38.7592599995931_-9.21870999998517
12 08 16 38.7592599995931 -9.21870999998517
200
